DO NOT RUN ALL! Contains Global Scenario

In [1]:
#Installing Libraries
!pip install emoji
!pip3 install googletrans==3.1.0a0
!pip install spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 12.8/12.8 MB 6.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
#Importing Libraries

import os
from tqdm import tqdm
import pandas as pd
import re
import emoji
from googletrans import Translator
import nltk
import spacy


In [3]:
#Define Clear Console function 
def clear_console():
    os.system('cls' if os.name == 'nt' else 'clear')

In [4]:
# Load the CSV file from local directory into a pandas DataFrame
df_tweets = pd.read_csv('D:/Office laptop/colabDatasets-20230408T073310Z-001/colabDatasets/Training data/Training.data.csv', encoding='latin-1', header = None)
df_tweets.columns=['Sentiment', 'id', 'Date', 'Query', 'User', 'Tweet']
df_tweets.head(1)

,Sentiment,id,Date,Query,User,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."


In [7]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 212188 to 1403939
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentiment  12000 non-null  int64 
 1   id         12000 non-null  int64 
 2   Date       12000 non-null  object
 3   Query      12000 non-null  object
 4   User       12000 non-null  object
 5   Tweet      12000 non-null  object
dtypes: int64(2), object(4)
memory usage: 656.2+ KB


In [8]:
# Rename the "Text" column to "Original Text"
df_tweets = df_tweets.rename(columns={'Tweet': 'Original Text'})

# Convert the "Original Text" column to a string data type
df_tweets['Original Text'] = df_tweets['Original Text'].astype(str)

# Create a duplicate column named "Text" with the values from the "Original Text" column
with tqdm(total=len(df_tweets.index), desc="Processing") as pbar:
    for i, row in df_tweets.iterrows():
        df_tweets.at[i, 'Text'] = row['Original Text']
        pbar.update(1)

print('Cell Complete')


Processing: 100%|█████████████████████████████████████████████████████████████| 12000/12000 [00:00<00:00, 13270.29it/s]

Cell Complete


Extract_mentions_hashtags uses regular expressions to extract mentions (@\w+) and hashtags (#\w+) from the text. 

In [9]:


def extract_mentions_hashtags(text):
    mentions = re.findall(r'@\w+', text)
    hashtags = re.findall(r'#\w+', text)
    for mention in mentions:
        text = text.replace(mention, '')
    for hashtag in hashtags:
        text = text.replace(hashtag, '')
    return ','.join(mentions), ','.join(hashtags), text

with tqdm(total=len(df_tweets.index), desc="Processing") as pbar:
    for i, row in df_tweets.iterrows():
        df_tweets.at[i, 'Mentions'], df_tweets.at[i, 'Hashtags'], df_tweets.at[i, 'Text'] = extract_mentions_hashtags(row['Text'])
        pbar.update(1)

print('Cell Complete')
df_tweets.info()


Processing: 100%|██████████████████████████████████████████████████████████████| 12000/12000 [00:01<00:00, 8813.81it/s]

Cell Complete
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 212188 to 1403939
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Sentiment      12000 non-null  int64 
 1   id             12000 non-null  int64 
 2   Date           12000 non-null  object
 3   Query          12000 non-null  object
 4   User           12000 non-null  object
 5   Original Text  12000 non-null  object
 6   Text           12000 non-null  object
 7   Mentions       12000 non-null  object
 8   Hashtags       12000 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.2+ MB


Define a function remove_urls that removes URLs from a given text using regular expressions. 

In [10]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)

with tqdm(total=len(df_tweets.index), desc="Processing") as pbar:
    df_tweets['URL Removed Text'] = df_tweets['Text'].apply(lambda x: remove_urls(x))
    pbar.update(1)

print('Cell Complete')
df_tweets.info()


Processing:   0%|                                                                    | 1/12000 [00:00<07:03, 28.31it/s]

Cell Complete
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 212188 to 1403939
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Sentiment         12000 non-null  int64 
 1   id                12000 non-null  int64 
 2   Date              12000 non-null  object
 3   Query             12000 non-null  object
 4   User              12000 non-null  object
 5   Original Text     12000 non-null  object
 6   Text              12000 non-null  object
 7   Mentions          12000 non-null  object
 8   Hashtags          12000 non-null  object
 9   URL Removed Text  12000 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.3+ MB


**Emoji Handling**
replaces emojis and emoticons with relevant words using the emoji 2.2.0 library:

The replace_emoji function uses the demojize method of the emoji library to convert emojis to their textual representation

In [11]:
def replace_emoji(text):
    return emoji.demojize(text).replace(":"," ")

with tqdm(total=len(df_tweets.index), desc="Processing") as pbar:
    df_tweets['Emoji Removed Text'] = df_tweets['URL Removed Text'].apply(lambda x: replace_emoji(x))
    pbar.update(1)
    
clear_console()
print('emoji Library Installed')
print('Cell Complete')
df_tweets.info()

Processing:   0%|                                                                    | 1/12000 [00:00<48:11,  4.15it/s]

emoji Library Installed
Cell Complete
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 212188 to 1403939
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Sentiment           12000 non-null  int64 
 1   id                  12000 non-null  int64 
 2   Date                12000 non-null  object
 3   Query               12000 non-null  object
 4   User                12000 non-null  object
 5   Original Text       12000 non-null  object
 6   Text                12000 non-null  object
 7   Mentions            12000 non-null  object
 8   Hashtags            12000 non-null  object
 9   URL Removed Text    12000 non-null  object
 10  Emoji Removed Text  12000 non-null  object
dtypes: int64(2), object(9)
memory usage: 1.4+ MB


Replace all underscores with spaces in the "text" column of the "df_tweets" dataframe.

In [12]:
df_tweets['Character Processed Text'] = df_tweets['Emoji Removed Text'].str.replace('_', ' ')
print('Cell Complete')
df_tweets.info()

Cell Complete
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 212188 to 1403939
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sentiment                 12000 non-null  int64 
 1   id                        12000 non-null  int64 
 2   Date                      12000 non-null  object
 3   Query                     12000 non-null  object
 4   User                      12000 non-null  object
 5   Original Text             12000 non-null  object
 6   Text                      12000 non-null  object
 7   Mentions                  12000 non-null  object
 8   Hashtags                  12000 non-null  object
 9   URL Removed Text          12000 non-null  object
 10  Emoji Removed Text        12000 non-null  object
 11  Character Processed Text  12000 non-null  object
dtypes: int64(2), object(10)
memory usage: 1.4+ MB


replaces multiple spaces with a single space in the "Text" column 

In [13]:
df_tweets['Character Processed Text'] = df_tweets['Character Processed Text'].str.replace(' +', ' ')
print('Cell Complete')
df_tweets.info()

C:\Users\sup3r\AppData\Local\Temp\ipykernel_15956\2964014356.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_tweets['Character Processed Text'] = df_tweets['Character Processed Text'].str.replace(' +', ' ')


Cell Complete
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 212188 to 1403939
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sentiment                 12000 non-null  int64 
 1   id                        12000 non-null  int64 
 2   Date                      12000 non-null  object
 3   Query                     12000 non-null  object
 4   User                      12000 non-null  object
 5   Original Text             12000 non-null  object
 6   Text                      12000 non-null  object
 7   Mentions                  12000 non-null  object
 8   Hashtags                  12000 non-null  object
 9   URL Removed Text          12000 non-null  object
 10  Emoji Removed Text        12000 non-null  object
 11  Character Processed Text  12000 non-null  object
dtypes: int64(2), object(10)
memory usage: 1.4+ MB


Remove non-alphanumeric characters from text in a DataFrame column using regex

In [14]:
def remove_non_alphanumeric(text):
    pattern = re.compile(r'[^a-zA-Z0-9 @\#]+')
    return pattern.sub('', text)

df_tweets['Character Processed Text'] = df_tweets['Character Processed Text'].apply(remove_non_alphanumeric)
print('Cell Complete')
df_tweets.info()

Cell Complete
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 212188 to 1403939
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sentiment                 12000 non-null  int64 
 1   id                        12000 non-null  int64 
 2   Date                      12000 non-null  object
 3   Query                     12000 non-null  object
 4   User                      12000 non-null  object
 5   Original Text             12000 non-null  object
 6   Text                      12000 non-null  object
 7   Mentions                  12000 non-null  object
 8   Hashtags                  12000 non-null  object
 9   URL Removed Text          12000 non-null  object
 10  Emoji Removed Text        12000 non-null  object
 11  Character Processed Text  12000 non-null  object
dtypes: int64(2), object(10)
memory usage: 1.4+ MB


str.lower() method

In [15]:
df_tweets['Character Processed Text'] = df_tweets['Character Processed Text'].str.lower()
print('Cell Complete')
df_tweets.info()

Cell Complete
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 212188 to 1403939
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sentiment                 12000 non-null  int64 
 1   id                        12000 non-null  int64 
 2   Date                      12000 non-null  object
 3   Query                     12000 non-null  object
 4   User                      12000 non-null  object
 5   Original Text             12000 non-null  object
 6   Text                      12000 non-null  object
 7   Mentions                  12000 non-null  object
 8   Hashtags                  12000 non-null  object
 9   URL Removed Text          12000 non-null  object
 10  Emoji Removed Text        12000 non-null  object
 11  Character Processed Text  12000 non-null  object
dtypes: int64(2), object(10)
memory usage: 1.4+ MB


Remove any rows in df_tweets where the "Text" column has only whitespaces or is empty after the above preprocessing steps

In [16]:
df_tweets = df_tweets[df_tweets['Character Processed Text'].astype(str).str.strip() != '']
print('Cell Complete')
df_tweets.info()

Cell Complete
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11969 entries, 212188 to 1403939
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sentiment                 11969 non-null  int64 
 1   id                        11969 non-null  int64 
 2   Date                      11969 non-null  object
 3   Query                     11969 non-null  object
 4   User                      11969 non-null  object
 5   Original Text             11969 non-null  object
 6   Text                      11969 non-null  object
 7   Mentions                  11969 non-null  object
 8   Hashtags                  11969 non-null  object
 9   URL Removed Text          11969 non-null  object
 10  Emoji Removed Text        11969 non-null  object
 11  Character Processed Text  11969 non-null  object
dtypes: int64(2), object(10)
memory usage: 1.2+ MB


In [17]:
# Translate the other languages to english
import googletrans
from googletrans import *
translator = googletrans.Translator()
print('Google Trans Imported')

df_tweets['Character Processed Text'] = df_tweets['Character Processed Text'].astype(str) # changing datatype to string

# Add a progress bar to show the translation progress
with tqdm(total=len(df_tweets)) as pbar:
    def translate_text(row):
        if detect(row['Character Processed Text']) != 'en':
            translated_text = translator.translate(row['Character Processed Text'], src='auto', dest='en').text
        else:
            translated_text = row['Character Processed Text']
        pbar.update(1)
        return translated_text

    df_tweets['Translated Text'] = df_tweets.apply(translate_text, axis=1)

print('Cell Complete')


Google Trans Imported


100%|████████████████████████████████████████████████████████████████████████████| 11969/11969 [42:29<00:00,  4.69it/s]

Cell Complete


In [18]:

# Download NLTK stop words
nltk.download('stopwords')

# Load stop words
stop_words = set(nltk.corpus.stopwords.words('english'))

# Define a function to remove stop words from text
def remove_stop_words(text):
    # Tokenize text
    words = nltk.word_tokenize(text)
    # Remove stop words
    words = [word for word in words if word.lower() not in stop_words]
    # Join words
    return ' '.join(words)

# Apply function to "Translated Text" column of DataFrame
df_tweets['Translated Text'] = df_tweets['Translated Text'].apply(remove_stop_words)

# Add progress bar using tqdm
for i, row in tqdm(df_tweets.iterrows(), total=df_tweets.shape[0]):
    df_tweets.at[i, 'Stopword Processed Text'] = remove_stop_words(row['Translated Text'])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sup3r\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████████████████████████████████████████████████████████████████████| 11969/11969 [00:02<00:00, 4411.82it/s]


In [23]:
# Load the English model
nlp = spacy.load('en_core_web_sm')

# Define a function to tokenize using spaCy
def tokenize(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

# Apply the tokenize function to the 'Text' column with progress bar
from tqdm import tqdm
tqdm.pandas()
df_tweets['Text_Tokenized'] = df_tweets['Stopword Processed Text'].progress_apply(tokenize)


100%|███████████████████████████████████████████████████████████████████████████| 11969/11969 [01:05<00:00, 183.77it/s]


In [26]:
# Load the English model
nlp = spacy.load('en_core_web_sm')

# Define a function to perform lemmatizing using spaCy
def lemmatize(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return lemmas

# Apply the lemmatize function to the 'Text' column with progress bar
from tqdm import tqdm
tqdm.pandas()
df_tweets['Text_Lemmatized'] = df_tweets['Stopword Processed Text'].progress_apply(lemmatize)


100%|███████████████████████████████████████████████████████████████████████████| 11969/11969 [01:05<00:00, 183.30it/s]


In [27]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11969 entries, 212188 to 1403939
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sentiment                 11969 non-null  int64 
 1   id                        11969 non-null  int64 
 2   Date                      11969 non-null  object
 3   Query                     11969 non-null  object
 4   User                      11969 non-null  object
 5   Original Text             11969 non-null  object
 6   Text                      11969 non-null  object
 7   Mentions                  11969 non-null  object
 8   Hashtags                  11969 non-null  object
 9   URL Removed Text          11969 non-null  object
 10  Emoji Removed Text        11969 non-null  object
 11  Character Processed Text  11969 non-null  object
 12  Translated Text           11969 non-null  object
 13  Stopword Processed Text   11969 non-null  object
 14  Text_Tokenized 

In [40]:
import pandas as pd
import csv


# Save the DataFrame to a CSV file
df_tweets.to_csv(r'D:/Office laptop/colabDatasets-20230408T073310Z-001/colabDatasets/Training data/.data.Preprocessed.csv', encoding='latin-1')